In [38]:

# import external libs
import pandas as pd
import warnings
from tqdm import tqdm
import os
import sys
sys.path.append('../src/')
import re
import plotly.express as px

# import internal libs
from model.evaluation import classification_report, regression_metrics, get_classification_report
from model.config import create_experiment_configs_dummy, create_experiment_configs_tf
from data.preparation import load_dataset

In [39]:
# remove warning
warnings.filterwarnings('ignore')

In [40]:
# define paths
PATH_REPORTS = '../reports/'
lstm_results_path = 'test_results/LSTM_with_Attention_{asset}_test_results.csv'
mlp_results_path =  'test_results/MLP_{asset}_test_results.csv'
DATA_DIR = '../data/'

In [41]:
# define experiments
ASSETS = [
    "PETR3.SA", 
    "PRIO3.SA", 
    "VALE3.SA", 
    "GGBR3.SA", 
    "ABCB4.SA", 
    "ITUB3.SA", 
    "FLRY3.SA", 
    "RADL3.SA"
    ]

seq_len_list = [1,2,3,4,5,6,7,14,21,28,35,42,49,56,63,70]

moving_windows = [7,14,21]

dict_experiments_dummy = create_experiment_configs_dummy(ASSETS, moving_windows)
dict_experiments_tf = create_experiment_configs_tf(ASSETS, seq_len_list, moving_windows)

## General results

### Create table

In [42]:
list_results_clf = []
list_results_reg = []
# list_results = []

for name, dict_experiments, path_results in [
    ("tf", dict_experiments_tf, PATH_REPORTS + 'test_results/{algorithm}_{asset}_features={features}__label={label_col}__sql_len={seq_len}__scaling_method={scaling_method}_test_results.csv'),
    ('dummy', dict_experiments_dummy, PATH_REPORTS + "test_results/Dummy_model_{asset}_features={feature_col}__label={label_col}_test_results.csv")
]:
    
    for exp_name, config in tqdm(dict_experiments.items()):
        
        if name == "tf":
            
            feature_cols = config['feature_cols']
            label_col = config['label_col']
            seq_len = config['seq_len']
            asset = config['asset']
            scaling_method = config['scaling_method']
            algorithm = config['algorithm']
            asset = config['asset']
            prediction_type = config['prediction_type']
            
            filepath = path_results.format(
                algorithm = algorithm,
                features = "_".join(feature_cols),
                label_col = label_col,
                asset = asset.replace(".", "_"),
                scaling_method = scaling_method.__str__(),
                seq_len = seq_len
            )
            
            if not os.path.exists(filepath): 
                print(f"The file {filepath} dont't exists")
                continue

            results = pd.read_csv(filepath)
            
        elif name == 'dummy':
            feature_cols = [config['feature_col']]
            label_col = config['label_col']
            asset = config['asset']
            algorithm = "Dummy_model"
            seq_len = 1
            scaling_method = None
            prediction_type = 'dummy'
            asset_formated = asset.replace(".", "_")
            
            filepath = path_results.format(
                algorithm = algorithm,
                feature_col = feature_cols[0],
                label_col = label_col,
                asset = asset_formated
            )
            
            if  not os.path.exists(filepath): 
                print(f"The file {filepath} dont't exists")
                continue
            
            results = pd.read_csv(filepath)
        
        # concat with the test dataset
        full_test = load_dataset(asset=asset, data_dir=DATA_DIR,dataset_split='test')
        results = pd.concat([results,full_test], axis =1)
        
        # find the window for label calculation (get the number of the string)
        window = int(re.findall(r'\d+', label_col)[0])

        # results without leak
        results_wo_leak = results.iloc[max(seq_len, window):]
        
        # remove invalid days
        results = results[results.Invalid_Days == 0]
        
        # raise error if there is nan values
        if results.isna().sum().sum() > 0: 
            raise ValueError('Há dados nulos no dataframe de resultados')
                
                
        new_coluns = [asset, str(feature_cols), str(label_col), seq_len, algorithm, scaling_method, prediction_type]
        new_columns_nms = ['asset','feature_cols','label_col','seq_len','model','scaling_method','prediction_type', 'leak']
                
        # regression metrics
        reg_metrics = regression_metrics(results.y_test, results.y_pred)
        reg_metrics_wo_leak = regression_metrics(results_wo_leak.y_test, results_wo_leak.y_pred)
        
    
        # add columns with experiment config 

        reg_metrics[
            new_columns_nms
            ] = new_coluns + [True]
        reg_metrics_wo_leak[
            new_columns_nms
            ] = new_coluns+ [False]
        
        list_results_reg.append(reg_metrics)
        list_results_reg.append(reg_metrics_wo_leak)
    
    
        # obtem metricas de classificacao, truncando os valores (se o valor já for a meta, não terá diferenca)
        y_test_trunc = [int(i) for i in results.y_test]
        y_pred_trunc = [int(i) for i in results.y_pred]
        df_cr = get_classification_report(y_test_trunc, y_pred_trunc)
        
        y_test_trunc_woleak = [int(i) for i in results_wo_leak.y_test]
        y_pred_trunc_woleak = [int(i) for i in results_wo_leak.y_pred]
        df_cr_wo_leak = get_classification_report(y_test_trunc_woleak, y_pred_trunc_woleak)
        
        df_cr[new_columns_nms] = new_coluns + [True]
        df_cr_wo_leak[new_columns_nms] = new_coluns + [False]
        
        list_results_clf.append(df_cr)
        list_results_clf.append(df_cr_wo_leak)

100%|██████████| 48/48 [00:02<00:00, 21.53it/s]


In [43]:
# concat results
final_results_reg = pd.concat(list_results_reg).reset_index(drop=True)
final_results_clf = pd.concat(list_results_clf).reset_index(drop=True)

In [44]:
# only the "right" labels
final_results_clf = final_results_clf[final_results_clf.label_col.str.contains('meta')]
final_results_reg = final_results_reg[final_results_reg.label_col.str.contains('diff_close_mean_z_score')]

### Results

In [45]:
assets_domain = [(["PETR3.SA","PRIO3.SA"], 'Petróleo'), (["VALE3.SA", "GGBR3.SA"], "Mineração"), (["ABCB4.SA", "ITUB3.SA"], 'Financeiro'), (["FLRY3.SA", "RADL3.SA"], 'Saúde')]

# Configuração para não omitir linhas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


#### Dummy results

In [46]:
assets_domain = [(["PETR3.SA","PRIO3.SA"], 'Petróleo'), (["VALE3.SA", "GGBR3.SA"], "Mineração"), (["ABCB4.SA", "ITUB3.SA"], 'Financeiro'), (["FLRY3.SA", "RADL3.SA"], 'Saúde')]

asset_to_domain = lambda x: 'Petróleo' if x in ["PETR3.SA","PRIO3.SA"] else "Mineração" if x in ["VALE3.SA", "GGBR3.SA"] else 'Financeiro' if x in ["ABCB4.SA", "ITUB3.SA"] else 'Saúde' if x in ["FLRY3.SA", "RADL3.SA"] else None

In [47]:
# general 
macro_dummy = final_results_clf[
    (final_results_clf['class'] == 'macro avg') &
    (final_results_clf['prediction_type'] == 'dummy')
    
    ].round(2)
macro_dummy['domain'] = macro_dummy.asset.apply(asset_to_domain)

reg_dummy = final_results_reg[
    (final_results_reg['prediction_type'] == 'dummy')
    ].round(2)
reg_dummy['domain'] = reg_dummy.asset.apply(asset_to_domain)

In [48]:
# for label in macro_dummy.label_col.unique():
#     for model in macro_dummy.model.unique():

#         print(label, model)
        
#         fig = px.box(macro_dummy, x="model", y="f1-score", points="all")
#         fig.show()

In [49]:
for assets, domain in assets_domain:
    print(f'''
##############################################
# DOMAIN: {domain}
# ASSETS: {assets}
##############################################
          ''')
    
    print('Metricas macro - label meta')
    metrics_clf = macro_dummy[macro_dummy.label_col.str.contains('meta') & macro_dummy.asset.isin(assets)].\
    drop(['support', 'scaling_method', 'prediction_type', 'seq_len', 'class', 'model'], axis =1 ).\
            rename({'asset': 'ativo', 'label_col': 'alvo'},axis=1)
        
        
    display(metrics_clf.sort_values(['alvo', 'leak', 'ativo']))
    
    print('Metricas regressao - label diff_close_mean_z_score')
    
    metrics_reg = reg_dummy[reg_dummy.label_col.str.contains('diff_close_mean_z_score') & reg_dummy.asset.isin(assets)].\
        drop(['scaling_method', 'prediction_type', 'seq_len', 'model', 'feature_cols'], axis = 1).\
            rename({'asset': 'ativo', 'label_col': 'alvo'},axis=1)
    display(metrics_reg)


##############################################
# DOMAIN: Petróleo
# ASSETS: ['PETR3.SA', 'PRIO3.SA']
##############################################
          
Metricas macro - label meta


,precision,recall,f1-score,ativo,feature_cols,alvo,leak,domain
39603,0.67,0.67,0.67,PETR3.SA,['past_meta_14'],meta_14,False,Petróleo
39704,0.65,0.65,0.65,PRIO3.SA,['past_meta_14'],meta_14,False,Petróleo
39594,0.43,0.43,0.43,PETR3.SA,['past_meta_14'],meta_14,True,Petróleo
39696,0.49,0.49,0.49,PRIO3.SA,['past_meta_14'],meta_14,True,Petróleo
39639,0.71,0.70,0.70,PETR3.SA,['past_meta_21'],meta_21,False,Petróleo
39735,0.74,0.74,0.74,PRIO3.SA,['past_meta_21'],meta_21,False,Petróleo
39630,0.49,0.49,0.49,PETR3.SA,['past_meta_21'],meta_21,True,Petróleo
39727,0.62,0.62,0.62,PRIO3.SA,['past_meta_21'],meta_21,True,Petróleo
39568,0.54,0.54,0.54,PETR3.SA,['past_meta_7'],meta_7,False,Petróleo
39672,0.48,0.48,0.48,PRIO3.SA,['past_meta_7'],meta_7,False,Petróleo


Metricas regressao - label diff_close_mean_z_score


,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE),ativo,alvo,leak,domain
4608,0.71,0.92,0.96,0.26,2.07,PETR3.SA,diff_close_mean_z_score_7,True,Petróleo
4609,0.49,0.63,0.79,0.48,1.42,PETR3.SA,diff_close_mean_z_score_7,False,Petróleo
4612,0.57,0.65,0.81,0.58,1.78,PETR3.SA,diff_close_mean_z_score_14,True,Petróleo
4613,0.39,0.45,0.67,0.71,1.23,PETR3.SA,diff_close_mean_z_score_14,False,Petróleo
4616,0.48,0.48,0.69,0.70,1.08,PETR3.SA,diff_close_mean_z_score_21,True,Petróleo
4617,0.33,0.33,0.58,0.79,0.74,PETR3.SA,diff_close_mean_z_score_21,False,Petróleo
4620,0.69,0.84,0.92,0.33,2.15,PRIO3.SA,diff_close_mean_z_score_7,True,Petróleo
4621,0.48,0.58,0.76,0.53,1.48,PRIO3.SA,diff_close_mean_z_score_7,False,Petróleo
4624,0.56,0.58,0.76,0.61,1.58,PRIO3.SA,diff_close_mean_z_score_14,True,Petróleo
4625,0.38,0.40,0.63,0.73,1.09,PRIO3.SA,diff_close_mean_z_score_14,False,Petróleo



##############################################
# DOMAIN: Mineração
# ASSETS: ['VALE3.SA', 'GGBR3.SA']
##############################################
          
Metricas macro - label meta


...

,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE),ativo,alvo,leak,domain
4656,0.76,1.00,1.00,0.17,2.658000e+02,ABCB4.SA,diff_close_mean_z_score_7,True,Financeiro
4657,0.52,0.68,0.82,0.44,1.827400e+02,ABCB4.SA,diff_close_mean_z_score_7,False,Financeiro
4660,0.62,0.71,0.84,0.50,2.740000e+00,ABCB4.SA,diff_close_mean_z_score_14,True,Financeiro
4661,0.43,0.49,0.70,0.66,1.900000e+00,ABCB4.SA,diff_close_mean_z_score_14,False,Financeiro
4664,0.54,0.52,0.72,0.66,1.690000e+00,ABCB4.SA,diff_close_mean_z_score_21,True,Financeiro
4665,0.37,0.36,0.60,0.77,1.170000e+00,ABCB4.SA,diff_close_mean_z_score_21,False,Financeiro
4668,0.68,0.88,0.94,0.33,4.948931e+12,ITUB3.SA,diff_close_mean_z_score_7,True,Financeiro
4669,0.46,0.59,0.77,0.54,3.402713e+12,ITUB3.SA,diff_close_mean_z_score_7,False,Financeiro
4672,0.52,0.55,0.74,0.67,1.679800e+02,ITUB3.SA,diff_close_mean_z_score_14,True,Financeiro
4673,0.36,0.37,0.61,0.78,1.163400e+02,ITUB3.SA,diff_close_mean_z_score_14,False,Financeiro



##############################################
# DOMAIN: Saúde
# ASSETS: ['FLRY3.SA', 'RADL3.SA']
##############################################
          
Metricas macro - label meta


,precision,recall,f1-score,ativo,feature_cols,alvo,leak,domain
40220,0.63,0.63,0.63,FLRY3.SA,['past_meta_14'],meta_14,False,Saúde
40320,0.66,0.66,0.66,RADL3.SA,['past_meta_14'],meta_14,False,Saúde
40210,0.38,0.38,0.38,FLRY3.SA,['past_meta_14'],meta_14,True,Saúde
40312,0.50,0.50,0.50,RADL3.SA,['past_meta_14'],meta_14,True,Saúde
40255,0.70,0.70,0.70,FLRY3.SA,['past_meta_21'],meta_21,False,Saúde
40356,0.61,0.61,0.61,RADL3.SA,['past_meta_21'],meta_21,False,Saúde
40247,0.48,0.48,0.48,FLRY3.SA,['past_meta_21'],meta_21,True,Saúde
40347,0.52,0.52,0.52,RADL3.SA,['past_meta_21'],meta_21,True,Saúde
40184,0.51,0.51,0.51,FLRY3.SA,['past_meta_7'],meta_7,False,Saúde
40288,0.50,0.50,0.50,RADL3.SA,['past_meta_7'],meta_7,False,Saúde


Metricas regressao - label diff_close_mean_z_score


,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE),ativo,alvo,leak,domain
4680,0.72,0.91,0.95,0.25,2.83,FLRY3.SA,diff_close_mean_z_score_7,True,Saúde
4681,0.49,0.62,0.79,0.48,1.94,FLRY3.SA,diff_close_mean_z_score_7,False,Saúde
4684,0.58,0.56,0.75,0.61,2.25,FLRY3.SA,diff_close_mean_z_score_14,True,Saúde
4685,0.39,0.39,0.62,0.74,1.56,FLRY3.SA,diff_close_mean_z_score_14,False,Saúde
4688,0.49,0.42,0.65,0.73,203.79,FLRY3.SA,diff_close_mean_z_score_21,True,Saúde
4689,0.34,0.29,0.54,0.81,142.19,FLRY3.SA,diff_close_mean_z_score_21,False,Saúde
4692,0.68,0.82,0.91,0.36,2.85,RADL3.SA,diff_close_mean_z_score_7,True,Saúde
4693,0.47,0.56,0.75,0.57,1.96,RADL3.SA,diff_close_mean_z_score_7,False,Saúde
4696,0.56,0.58,0.76,0.62,3164.58,RADL3.SA,diff_close_mean_z_score_14,True,Saúde
4697,0.39,0.40,0.63,0.74,2191.88,RADL3.SA,diff_close_mean_z_score_14,False,Saúde


In [50]:
for leak in [True, False]:
        
    fig = px.box(
        macro_dummy[macro_dummy.leak == leak], x="label_col", 
        y="f1-score", 
        points="all", 
        title = f'Distribuição de resultados para cada alvo com leak={leak} | model=dummy')
    
    fig.write_image(PATH_REPORTS + f"/images/box_plot_alvo_dummy_leak={leak}_clf.png")
    fig.show()

for leak in [True, False]:
        
    fig = px.box(
        reg_dummy[reg_dummy.leak == leak], x="label_col", 
        y="R-squared (R2)", 
        points="all", 
        title = f'Distribuição de resultados para cada alvo com leak={leak} | model=dummy')
    
    fig.write_image(PATH_REPORTS + f"/images/box_plot_alvo_dummy_leak={leak}_reg.png")
    fig.show()

In [51]:
for leak in [True, False]:
        
    fig = px.box(
        macro_dummy[macro_dummy.leak == leak], x="asset", 
        y="f1-score", 
        points="all",
        color = 'domain',
        title = f'Distribuição de resultados para cada ativo com leak={leak}')
    
    fig.write_image(PATH_REPORTS + f"/images/box_plot_asset_dummy_leak={leak}_clf.png")
    fig.show()

for leak in [True, False]:
        
    fig = px.box(
        reg_dummy[reg_dummy.leak == leak], x="asset", 
        y="R-squared (R2)", 
        points="all", 
        color = 'domain',
        title = f'Distribuição de resultados para cada ativo com leak={leak}')
    
    fig.write_image(PATH_REPORTS + f"/images/box_plot_asset_dummy_leak={leak}_ref.png")
    fig.show()

#### NN results

In [52]:
# general 
macro_clf = final_results_clf[(final_results_clf['class'] == 'macro avg')].round(5)
reg= final_results_reg.round(5)

In [53]:
for label in reg.label_col.unique():

    for leak in [False]:

        print(f"label: {label} - leak: {leak}")
        
        
        if 'meta' in label: label_formated = label.replace('meta', 'k')
        if 'diff_close_mean_z_score' in label: label_formated = label.replace('diff_close_mean_z_score', 'z')
        
        df_plt = reg[(reg.label_col == label) & (reg.leak == leak)]
        
        fig = px.box(
            df_plt, 
            x="model", 
            y="R-squared (R2)", 
            points="all",
            title = f'Distribuição de f1 para os experimentos \n label:{label_formated}|leak={leak}'
            )
        
        fig.write_image(PATH_REPORTS + f"/images/box_plot_exp_dist_label={label_formated}__leak={leak}_reg.png")
        # fig.update_traces(boxpoints=False) 
        fig.show()

label: diff_close_mean_z_score_7 - leak: False


label: diff_close_mean_z_score_14 - leak: False


label: diff_close_mean_z_score_21 - leak: False


In [54]:
for label in macro_clf.label_col.unique():

    for leak in [False]:

        print(f"label: {label} - leak: {leak}")
        
        if 'meta' in label: label_col = label.replace('meta', 'k')
        if 'diff_close_mean_z_score' in label: label_col = label.replace('diff_close_mean_z_score', 'z')
        
        df_plt = macro_clf[(macro_clf.label_col == label) & (macro_clf.leak == leak)]
        
        fig = px.box(
            df_plt, 
            x="model", 
            y="f1-score", 
            points="all",
            title = f'Distribuição dos resultados de f1 para os experimentos - label: {label} | leak={leak}'
            )
        
        fig.write_image(PATH_REPORTS + f"/images/box_plot_exp_dist_label={label}__leak={leak}_clf.png")
        fig.show()

label: meta_7 - leak: False


label: meta_14 - leak: False


label: meta_21 - leak: False


In [55]:
for assets, domain in assets_domain:
    print(f'''
##############################################
# DOMAIN: {domain}
# ASSETS: {assets}
##############################################
          ''')
    
    print('Metricas macro - label meta')
    metrics_clf = macro_clf[macro_clf.label_col.str.contains('meta') & macro_clf.asset.isin(assets)].\
    drop(['support', 'scaling_method', 'prediction_type', 'class'], axis =1 ).\
            rename({'asset': 'ativo', 'label_col': 'alvo'},axis=1)[['ativo','seq_len','alvo','precision','recall', 'f1-score', 'model', 'leak', 'feature_cols']]
    
    
    display(metrics_clf.loc[metrics_clf.groupby(['alvo','ativo', 'leak'])['f1-score'].idxmax()])
    
    print('Metricas regressao - label diff_close_mean_z_score')
    
    metrics_reg = reg[reg.label_col.str.contains('diff_close_mean_z_score') & reg.asset.isin(assets)].\
        drop(['scaling_method', 'prediction_type', 'feature_cols'], axis = 1).\
            rename({'asset': 'ativo', 'label_col': 'alvo','f1-score':'valor'},axis=1)
    
    display(metrics_reg.loc[metrics_reg.groupby(['alvo','ativo', 'leak'])['R-squared (R2)'].idxmax()])


##############################################
# DOMAIN: Petróleo
# ASSETS: ['PETR3.SA', 'PRIO3.SA']
##############################################
          
Metricas macro - label meta


,ativo,seq_len,alvo,precision,recall,f1-score,model,leak,feature_cols
39603,PETR3.SA,1,meta_14,0.66893,0.66658,0.66773,Dummy_model,False,['past_meta_14']
101,PETR3.SA,1,meta_14,0.43912,0.44128,0.44018,LSTM_with_Attention,True,['meta_14']
15227,PRIO3.SA,7,meta_14,0.66826,0.64541,0.65540,MLP,False,['meta_14']
15219,PRIO3.SA,7,meta_14,0.50495,0.48843,0.49515,MLP,True,['meta_14']
39639,PETR3.SA,1,meta_21,0.70554,0.70281,0.70415,Dummy_model,False,['past_meta_21']
12556,PETR3.SA,6,meta_21,0.51268,0.49143,0.50011,MLP,True,['meta_21']
522,PRIO3.SA,1,meta_21,0.74257,0.74321,0.74289,LSTM_with_Attention,False,['meta_21']
514,PRIO3.SA,1,meta_21,0.62055,0.62055,0.62055,LSTM_with_Attention,True,['meta_21']
39568,PETR3.SA,1,meta_7,0.53916,0.53916,0.53916,Dummy_model,False,['past_meta_7']
22207,PETR3.SA,28,meta_7,0.33797,0.34697,0.34241,LSTM_with_Attention,True,['meta_7']


Metricas regressao - label diff_close_mean_z_score


,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE),ativo,alvo,seq_len,model,leak
21,0.42061,0.42128,0.64906,0.72737,1.13465,PETR3.SA,diff_close_mean_z_score_14,1,MLP,False
596,0.53242,0.58694,0.76612,0.61799,1.49762,PETR3.SA,diff_close_mean_z_score_14,3,MLP,True
921,0.42128,0.37641,0.61352,0.74747,1.06357,PRIO3.SA,diff_close_mean_z_score_14,4,MLP,False
920,0.52801,0.52085,0.72170,0.64911,1.43553,PRIO3.SA,diff_close_mean_z_score_14,4,MLP,True
33,0.35096,0.31627,0.56238,0.80074,0.77326,PETR3.SA,diff_close_mean_z_score_21,1,MLP,False
32,0.45601,0.44708,0.66864,0.72368,0.99606,PETR3.SA,diff_close_mean_z_score_21,1,MLP,True
69,0.35600,0.28617,0.53495,0.82653,0.93495,PRIO3.SA,diff_close_mean_z_score_21,1,MLP,False
68,0.46039,0.40088,0.63315,0.75425,1.27224,PRIO3.SA,diff_close_mean_z_score_21,1,MLP,True
585,0.51527,0.54541,0.73852,0.54907,1.27261,PETR3.SA,diff_close_mean_z_score_7,3,MLP,False
2310,0.66265,0.74900,0.86545,0.39481,1.40114,PETR3.SA,diff_close_mean_z_score_7,21,LSTM_with_Attention,True



##############################################
# DOMAIN: Mineração
# ASSETS: ['VALE3.SA', 'GGBR3.SA']
##############################################
          
Metricas macro - label meta


...

,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE),ativo,alvo,seq_len,model,leak
1029,0.46039,0.44700,0.66858,0.68662,1.770360e+00,ABCB4.SA,diff_close_mean_z_score_14,4,MLP,False
162,0.59619,0.62396,0.78991,0.56245,2.071310e+00,ABCB4.SA,diff_close_mean_z_score_14,1,LSTM_with_Attention,True
201,0.40605,0.35764,0.59803,0.78431,1.062147e+02,ITUB3.SA,diff_close_mean_z_score_14,1,MLP,False
200,0.50626,0.49884,0.70629,0.69884,1.532851e+02,ITUB3.SA,diff_close_mean_z_score_14,1,MLP,True
177,0.39372,0.34028,0.58333,0.78025,1.189840e+00,ABCB4.SA,diff_close_mean_z_score_21,1,MLP,False
176,0.51252,0.48195,0.69423,0.68704,1.590710e+00,ABCB4.SA,diff_close_mean_z_score_21,1,MLP,True
4677,0.29513,0.26442,0.51422,0.85552,1.936570e+00,ITUB3.SA,diff_close_mean_z_score_21,1,Dummy_model,False
212,0.42709,0.37301,0.61074,0.79576,2.587100e+00,ITUB3.SA,diff_close_mean_z_score_21,1,MLP,True
1593,0.55276,0.57438,0.75788,0.52193,1.296199e+02,ABCB4.SA,diff_close_mean_z_score_7,6,MLP,False
1878,0.69537,0.78788,0.88762,0.34793,1.530098e+02,ABCB4.SA,diff_close_mean_z_score_7,7,LSTM_with_Attention,True



##############################################
# DOMAIN: Saúde
# ASSETS: ['FLRY3.SA', 'RADL3.SA']
##############################################
          
Metricas macro - label meta


,ativo,seq_len,alvo,precision,recall,f1-score,model,leak,feature_cols
40220,FLRY3.SA,1,meta_14,0.63270,0.63292,0.63281,Dummy_model,False,['past_meta_14']
14279,FLRY3.SA,6,meta_14,0.41068,0.38572,0.39369,LSTM_with_Attention,True,['meta_14']
4741,RADL3.SA,2,meta_14,0.66935,0.65568,0.66191,LSTM_with_Attention,False,['meta_14']
22003,RADL3.SA,21,meta_14,0.54020,0.49680,0.51122,LSTM_with_Attention,True,['meta_14']
40255,FLRY3.SA,1,meta_21,0.69998,0.70159,0.70078,Dummy_model,False,['past_meta_21']
34195,FLRY3.SA,56,meta_21,0.52664,0.47592,0.49440,MLP,True,['meta_21']
2378,RADL3.SA,1,meta_21,0.61627,0.62320,0.61961,LSTM_with_Attention,False,['meta_21']
2368,RADL3.SA,1,meta_21,0.52498,0.53231,0.52847,LSTM_with_Attention,True,['meta_21']
9291,FLRY3.SA,4,meta_7,0.70839,0.48968,0.52261,KAN,False,['meta_7']
9267,FLRY3.SA,4,meta_7,0.33232,0.34073,0.33644,MLP,True,['meta_7']


Metricas regressao - label diff_close_mean_z_score


,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE),ativo,alvo,seq_len,model,leak
237,0.42929,0.36396,0.60329,0.75248,1.44920,FLRY3.SA,diff_close_mean_z_score_14,1,MLP,False
236,0.54371,0.50705,0.71207,0.65219,2.02373,FLRY3.SA,diff_close_mean_z_score_14,1,MLP,True
273,0.41696,0.37753,0.61444,0.75685,1978.84622,RADL3.SA,diff_close_mean_z_score_14,1,MLP,False
848,0.53352,0.52831,0.72685,0.65624,2365.65046,RADL3.SA,diff_close_mean_z_score_14,3,MLP,True
249,0.37334,0.28285,0.53184,0.81967,340.72122,FLRY3.SA,diff_close_mean_z_score_21,1,MLP,False
536,0.47316,0.39225,0.62630,0.74608,71.38017,FLRY3.SA,diff_close_mean_z_score_21,2,MLP,True
285,0.35329,0.28109,0.53018,0.82624,2.79281,RADL3.SA,diff_close_mean_z_score_21,1,MLP,False
284,0.46245,0.39842,0.63121,0.75261,3.98159,RADL3.SA,diff_close_mean_z_score_21,1,MLP,True
513,0.53640,0.54470,0.73804,0.54997,1.61485,FLRY3.SA,diff_close_mean_z_score_7,2,MLP,False
2240,0.66824,0.73317,0.85625,0.39484,2.02946,FLRY3.SA,diff_close_mean_z_score_7,14,MLP,True


In [56]:

print('Metricas macro - label meta')
metrics_clf = macro_clf[macro_clf.label_col.str.contains('meta')].\
drop(['support', 'scaling_method', 'prediction_type', 'class'], axis =1 ).\
        rename({'asset': 'ativo', 'label_col': 'alvo'},axis=1)[['ativo','seq_len','alvo', 'f1-score', 'model', 'leak']]

metrics_clf = metrics_clf[metrics_clf.leak == False]

metrics_clf.alvo = metrics_clf.alvo.apply(lambda x: x.replace('meta', 'k'))

display(metrics_clf.loc[metrics_clf.groupby(['alvo','ativo', 'leak'])['f1-score'].idxmax()].drop('leak', axis = 1).round(2))

print('Metricas regressao - label diff_close_mean_z_score')


metrics_reg = reg[reg.label_col.str.contains('diff_close_mean_z_score')].\
    drop(['scaling_method', 'prediction_type', 'feature_cols'], axis = 1).\
        rename({'asset': 'ativo', 'label_col': 'alvo','f1-score':'valor'},axis=1)[['ativo','seq_len','alvo', 'R-squared (R2)', 'model', 'leak']].round(2)
        
metrics_reg = metrics_reg[metrics_reg.leak == False]

metrics_reg.alvo = metrics_reg.alvo.apply(lambda x: x.replace('diff_close_mean_z_score', 'z'))

display(metrics_reg.loc[metrics_reg.groupby(['alvo','ativo', 'leak'])['R-squared (R2)'].idxmax()])

Metricas macro - label meta


,ativo,seq_len,alvo,f1-score,model
6281,ABCB4.SA,3,k_14,0.66,MLP
40220,FLRY3.SA,1,k_14,0.63,Dummy_model
39903,GGBR3.SA,1,k_14,0.69,Dummy_model
1643,ITUB3.SA,1,k_14,0.58,LSTM_with_Attention
39603,PETR3.SA,1,k_14,0.67,Dummy_model
15227,PRIO3.SA,7,k_14,0.66,MLP
4741,RADL3.SA,2,k_14,0.66,LSTM_with_Attention
39800,VALE3.SA,1,k_14,0.70,Dummy_model
21205,ABCB4.SA,21,k_21,0.62,KAN
40255,FLRY3.SA,1,k_21,0.70,Dummy_model


Metricas regressao - label diff_close_mean_z_score


,ativo,seq_len,alvo,R-squared (R2),model,leak
165,ABCB4.SA,1,z_14,0.69,MLP,False
237,FLRY3.SA,1,z_14,0.75,MLP,False
129,GGBR3.SA,1,z_14,0.76,MLP,False
201,ITUB3.SA,1,z_14,0.78,MLP,False
21,PETR3.SA,1,z_14,0.73,MLP,False
57,PRIO3.SA,1,z_14,0.75,MLP,False
273,RADL3.SA,1,z_14,0.76,MLP,False
93,VALE3.SA,1,z_14,0.79,MLP,False
177,ABCB4.SA,1,z_21,0.78,MLP,False
249,FLRY3.SA,1,z_21,0.82,MLP,False


In [57]:
for assets, domain in assets_domain:
    print(f'''
##############################################
# DOMAIN: {domain}
# ASSETS: {assets}
##############################################
          ''')
    
    print('Metricas macro - label meta')
    metrics_clf = macro_clf[macro_clf.label_col.str.contains('meta') & macro_clf.asset.isin(assets)].\
    drop(['support', 'scaling_method', 'prediction_type', 'class'], axis =1 ).\
            rename({'asset': 'ativo', 'label_col': 'alvo'},axis=1)[['ativo','seq_len','alvo','precision','recall', 'f1-score', 'model', 'leak', 'feature_cols']]
    
    display(metrics_clf.loc[metrics_clf.groupby(['alvo','ativo', 'leak'])['f1-score'].idxmax()])
    
    print('Metricas regressao - label diff_close_mean_z_score')
    
    
    metrics_reg = reg[reg.label_col.str.contains('diff_close_mean_z_score') & reg.asset.isin(assets)].\
        drop(['scaling_method', 'prediction_type', 'feature_cols'], axis = 1).\
            rename({'asset': 'ativo', 'label_col': 'alvo','f1-score':'valor'},axis=1)
    
    display(metrics_reg.loc[metrics_reg.groupby(['alvo','ativo', 'leak'])['R-squared (R2)'].idxmax()])


##############################################
# DOMAIN: Petróleo
# ASSETS: ['PETR3.SA', 'PRIO3.SA']
##############################################
          
Metricas macro - label meta


,ativo,seq_len,alvo,precision,recall,f1-score,model,leak,feature_cols
39603,PETR3.SA,1,meta_14,0.66893,0.66658,0.66773,Dummy_model,False,['past_meta_14']
101,PETR3.SA,1,meta_14,0.43912,0.44128,0.44018,LSTM_with_Attention,True,['meta_14']
15227,PRIO3.SA,7,meta_14,0.66826,0.64541,0.65540,MLP,False,['meta_14']
15219,PRIO3.SA,7,meta_14,0.50495,0.48843,0.49515,MLP,True,['meta_14']
39639,PETR3.SA,1,meta_21,0.70554,0.70281,0.70415,Dummy_model,False,['past_meta_21']
12556,PETR3.SA,6,meta_21,0.51268,0.49143,0.50011,MLP,True,['meta_21']
522,PRIO3.SA,1,meta_21,0.74257,0.74321,0.74289,LSTM_with_Attention,False,['meta_21']
514,PRIO3.SA,1,meta_21,0.62055,0.62055,0.62055,LSTM_with_Attention,True,['meta_21']
39568,PETR3.SA,1,meta_7,0.53916,0.53916,0.53916,Dummy_model,False,['past_meta_7']
22207,PETR3.SA,28,meta_7,0.33797,0.34697,0.34241,LSTM_with_Attention,True,['meta_7']


Metricas regressao - label diff_close_mean_z_score


,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE),ativo,alvo,seq_len,model,leak
21,0.42061,0.42128,0.64906,0.72737,1.13465,PETR3.SA,diff_close_mean_z_score_14,1,MLP,False
596,0.53242,0.58694,0.76612,0.61799,1.49762,PETR3.SA,diff_close_mean_z_score_14,3,MLP,True
921,0.42128,0.37641,0.61352,0.74747,1.06357,PRIO3.SA,diff_close_mean_z_score_14,4,MLP,False
920,0.52801,0.52085,0.72170,0.64911,1.43553,PRIO3.SA,diff_close_mean_z_score_14,4,MLP,True
33,0.35096,0.31627,0.56238,0.80074,0.77326,PETR3.SA,diff_close_mean_z_score_21,1,MLP,False
32,0.45601,0.44708,0.66864,0.72368,0.99606,PETR3.SA,diff_close_mean_z_score_21,1,MLP,True
69,0.35600,0.28617,0.53495,0.82653,0.93495,PRIO3.SA,diff_close_mean_z_score_21,1,MLP,False
68,0.46039,0.40088,0.63315,0.75425,1.27224,PRIO3.SA,diff_close_mean_z_score_21,1,MLP,True
585,0.51527,0.54541,0.73852,0.54907,1.27261,PETR3.SA,diff_close_mean_z_score_7,3,MLP,False
2310,0.66265,0.74900,0.86545,0.39481,1.40114,PETR3.SA,diff_close_mean_z_score_7,21,LSTM_with_Attention,True



##############################################
# DOMAIN: Mineração
# ASSETS: ['VALE3.SA', 'GGBR3.SA']
##############################################
          
Metricas macro - label meta


...

,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE),ativo,alvo,seq_len,model,leak
1029,0.46039,0.44700,0.66858,0.68662,1.770360e+00,ABCB4.SA,diff_close_mean_z_score_14,4,MLP,False
162,0.59619,0.62396,0.78991,0.56245,2.071310e+00,ABCB4.SA,diff_close_mean_z_score_14,1,LSTM_with_Attention,True
201,0.40605,0.35764,0.59803,0.78431,1.062147e+02,ITUB3.SA,diff_close_mean_z_score_14,1,MLP,False
200,0.50626,0.49884,0.70629,0.69884,1.532851e+02,ITUB3.SA,diff_close_mean_z_score_14,1,MLP,True
177,0.39372,0.34028,0.58333,0.78025,1.189840e+00,ABCB4.SA,diff_close_mean_z_score_21,1,MLP,False
176,0.51252,0.48195,0.69423,0.68704,1.590710e+00,ABCB4.SA,diff_close_mean_z_score_21,1,MLP,True
4677,0.29513,0.26442,0.51422,0.85552,1.936570e+00,ITUB3.SA,diff_close_mean_z_score_21,1,Dummy_model,False
212,0.42709,0.37301,0.61074,0.79576,2.587100e+00,ITUB3.SA,diff_close_mean_z_score_21,1,MLP,True
1593,0.55276,0.57438,0.75788,0.52193,1.296199e+02,ABCB4.SA,diff_close_mean_z_score_7,6,MLP,False
1878,0.69537,0.78788,0.88762,0.34793,1.530098e+02,ABCB4.SA,diff_close_mean_z_score_7,7,LSTM_with_Attention,True



##############################################
# DOMAIN: Saúde
# ASSETS: ['FLRY3.SA', 'RADL3.SA']
##############################################
          
Metricas macro - label meta


,ativo,seq_len,alvo,precision,recall,f1-score,model,leak,feature_cols
40220,FLRY3.SA,1,meta_14,0.63270,0.63292,0.63281,Dummy_model,False,['past_meta_14']
14279,FLRY3.SA,6,meta_14,0.41068,0.38572,0.39369,LSTM_with_Attention,True,['meta_14']
4741,RADL3.SA,2,meta_14,0.66935,0.65568,0.66191,LSTM_with_Attention,False,['meta_14']
22003,RADL3.SA,21,meta_14,0.54020,0.49680,0.51122,LSTM_with_Attention,True,['meta_14']
40255,FLRY3.SA,1,meta_21,0.69998,0.70159,0.70078,Dummy_model,False,['past_meta_21']
34195,FLRY3.SA,56,meta_21,0.52664,0.47592,0.49440,MLP,True,['meta_21']
2378,RADL3.SA,1,meta_21,0.61627,0.62320,0.61961,LSTM_with_Attention,False,['meta_21']
2368,RADL3.SA,1,meta_21,0.52498,0.53231,0.52847,LSTM_with_Attention,True,['meta_21']
9291,FLRY3.SA,4,meta_7,0.70839,0.48968,0.52261,KAN,False,['meta_7']
9267,FLRY3.SA,4,meta_7,0.33232,0.34073,0.33644,MLP,True,['meta_7']


Metricas regressao - label diff_close_mean_z_score


,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE),ativo,alvo,seq_len,model,leak
237,0.42929,0.36396,0.60329,0.75248,1.44920,FLRY3.SA,diff_close_mean_z_score_14,1,MLP,False
236,0.54371,0.50705,0.71207,0.65219,2.02373,FLRY3.SA,diff_close_mean_z_score_14,1,MLP,True
273,0.41696,0.37753,0.61444,0.75685,1978.84622,RADL3.SA,diff_close_mean_z_score_14,1,MLP,False
848,0.53352,0.52831,0.72685,0.65624,2365.65046,RADL3.SA,diff_close_mean_z_score_14,3,MLP,True
249,0.37334,0.28285,0.53184,0.81967,340.72122,FLRY3.SA,diff_close_mean_z_score_21,1,MLP,False
536,0.47316,0.39225,0.62630,0.74608,71.38017,FLRY3.SA,diff_close_mean_z_score_21,2,MLP,True
285,0.35329,0.28109,0.53018,0.82624,2.79281,RADL3.SA,diff_close_mean_z_score_21,1,MLP,False
284,0.46245,0.39842,0.63121,0.75261,3.98159,RADL3.SA,diff_close_mean_z_score_21,1,MLP,True
513,0.53640,0.54470,0.73804,0.54997,1.61485,FLRY3.SA,diff_close_mean_z_score_7,2,MLP,False
2240,0.66824,0.73317,0.85625,0.39484,2.02946,FLRY3.SA,diff_close_mean_z_score_7,14,MLP,True


In [58]:
for leak in [True, False]:
        
    fig = px.box(
        macro_clf[macro_clf.leak == leak], x="label_col", 
        y="f1-score", 
        points="all", 
        title = f'Distribuição de resultados para cada alvo com leak={leak}')
    
    fig.write_image(PATH_REPORTS + f"/images/box_plot_alvo_models_leak={leak}_clf.png")
    fig.show()

for leak in [True, False]:
        
    fig = px.box(
        reg[reg.leak == leak], x="label_col", 
        y="R-squared (R2)", 
        points="all", 
        title = f'Distribuição de resultados para cada alvo com leak={leak}')
    
    fig.write_image(PATH_REPORTS + f"/images/box_plot_alvo_models_leak={leak}_reg.png")
    fig.show()

In [59]:

macro_clf['domain'] = macro_clf.asset.apply(asset_to_domain)
reg['domain'] = reg.asset.apply(asset_to_domain)

for leak in [True, False]:
        
    fig = px.box(
        macro_clf[macro_clf.leak == leak], x="asset", 
        y="f1-score", 
        points="all", 
        color = 'domain',
        title = f'Distribuição de resultados para cada ativo com leak={leak}')
    
    fig.write_image(PATH_REPORTS + f"/images/box_plot_asset_models_leak={leak}_clf.png")
    fig.show()

for leak in [True, False]:
    
    fig = px.box(
        reg[reg.leak == leak], x="asset", 
        y="R-squared (R2)", 
        points="all", 
        color = 'domain',
        title = f'Distribuição de resultados para cada ativo com leak={leak}')
    
    fig.write_image(PATH_REPORTS + f"/images/box_plot_asset_models_leak={leak}_reg.png")
    fig.show()

In [60]:
# for assets, domain in assets_domain:
#     print(f'''
# ##############################################
# # DOMAIN: {domain}
# # ASSETS: {assets}
# ##############################################
#           ''')
    
#     print('Metricas macro - label meta')
#     metrics_clf = macro_clf[macro_clf.label_col.str.contains('meta') & macro_clf.asset.isin(assets)].\
#     drop(['support', 'scaling_method', 'prediction_type', 'class', 'feature_cols'], axis =1 ).\
#             rename({'asset': 'ativo', 'label_col': 'alvo'},axis=1)[['ativo','seq_len','alvo','precision','recall', 'f1-score', 'model', 'leak']]
    
#     display(metrics_clf)
    
#     print('Metricas regressao - label diff_close_mean_z_score')
    
    
#     metrics_reg = reg[reg.label_col.str.contains('diff_close_mean_z_score') & reg.asset.isin(assets)].\
#         drop(['scaling_method', 'prediction_type', 'feature_cols'], axis = 1).\
#             rename({'asset': 'ativo', 'label_col': 'alvo','f1-score':'valor'},axis=1)
#     display(metrics_reg)